In [69]:
from __future__ import print_function
from rdkit import Chem
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import gzip
import multiprocessing as mp



first we want to load the data:

- ~/Documents/git/DUDE/DUDE contains a folder for each of the 102 targets in DUDE
- each folder contains *.ism / *.mol2 / *.sdf files for active and decoy molecules 
- to try to predict "bioactivity" (althouhg, how do we know that decoys are actually not active on other targers? <span style="color:red">check how exactly decoys are genereated</span>



In [110]:
#get the 102 directors and extract the data 


path = "../DUDE/DUDE"
#get all folders
all_targ = os.listdir(path)
#output = mp.Queue()

#for 102 example..... get *sdf.gz


def make_dframe(name):
    tmp_act = gzip.open(path+"/"+name+"/"+'actives_final.sdf.gz')
    tmp_dec = gzip.open(path+"/"+name+"/"+'decoys_final.sdf.gz')
    
    #read files
    cont_act = Chem.ForwardSDMolSupplier(tmp_act)
    cont_dec = Chem.ForwardSDMolSupplier(tmp_dec)

    #and make a list
    list_act = [x for x in cont_act if x is not None]
    list_dec = [x for x in cont_dec if x is not None]

    from rdkit.Chem import AllChem
    dframe = pd.DataFrame(columns=['target_name',"fp","active"]) 
    
    for m in list_act:
        dframe = dframe.append({'target_name':name,
                                'fp':AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=2048),
                               'active':1},ignore_index=True)
    for m in list_dec:
        dframe = dframe.append({'target_name':name,
                                'fp':AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=2048),
                                'active':0}, ignore_index=True)
    #output.put(dframe)
    return dframe

In [111]:
pool = mp.Pool(35)

result = pool.map(make_dframe, [x for x in all_targ]) 


In [113]:
dframe = pd.DataFrame(columns=['target_name',"fp","active"]) 
for df in result:
    dframe = dframe.append(df)
    


now that we have a pandas dataframe with all the morgan fingerpritns of all the active and decoys of DUDE we build a very high lvl TF Pipeline